
# 自動機械学習 Automated Machine Learning による自動車価格予測モデリング & モデル解釈 (リモート高速実行)


Azure Machine Learning の Automated ML を用いて、中古車の属性データから価格を予測するモデルを構築します。

1. 事前準備
    - Python SDK のインポート
    - Azure ML `Workspace` への接続
    - `Experiment` の作成
    - `Dataset` の作成と登録


2. 自動機械学習 Automated Machine Learning
    - 計算環境 `Machine Learning Compute` の準備
    - 自動機械学習 Automated ML の事前設定
    - モデル学習と結果の確認


3. モデル解釈


## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning の Python SDKをインポートします。

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

Azure Machine Learning Python SDK のバージョンを確認します。

In [ ]:
print(azureml.core.VERSION)

In [ ]:
# その他必要なライブラリを利用します
import logging
from matplotlib import pyplot as plt
import pandas as pd
import os
import warnings

# 警告メッセージを削除する
warnings.showwarning = lambda *args, **kwargs: None

### Azure ML workspace との接続
Azure Machine Learning との接続を行います。Azure Active Directory の認証が必要です。

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

### 実験名の設定

Azure Machine Learing では 実験を管理する仕組みがあります。自動機械学習は自動的にその実験管理の仕組みでメトリックやログが残ります。

In [ ]:
experiment_name = 'automl-reg-automobile-remote-explain'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

Azure Machine Learning service の計算環境 (Machine Learning Compute) で学習を回すために、Azure Machine Learning service の Dataset のフォーマットでデータを定義します。

automobile のデータは [automobile.csv](https://raw.githubusercontent.com/konabuta/Automated-ML-Workshop/master/data/automobile.csv) のリンクから入手できます。

In [ ]:
dataset = Dataset.get_by_name(ws, name='automobile')
dataset.take(5).to_pandas_dataframe()

In [ ]:
#予測変数の指定
label = 'price'

### 学習データとテストデータに分割

In [ ]:
train_dataset, test_dataset = dataset.random_split(0.8, seed=1234)

### Dataset として登録
Azure Machine Learning の データセット管理機能 Dataset に登録します。

In [ ]:
train_dataset = train_dataset.register(workspace = ws, name = 'automobile-train', description = 'automobile train workshop', create_new_version=True)
test_dataset = test_dataset.register(workspace = ws, name = 'automobile-test', description = 'automobile train workshop', create_new_version=True)

### 計算環境 (Machine Learning Compute) の設定

Azure Machine Learning では機械学習のための計算環境 Machine Learning Compute が利用できます。

In [ ]:
# 予め Azure ML studio にて cpucluster という名称の Machine Learning Compute を 作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(ws, "cpucluster")

## 2. 自動機械学習 Automated Machine Learning
自動機械学習の事前設定を行います。[AutoMLConfig](https://docs.microsoft.com/ja-JP/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) を用いて定義します。

In [ ]:
automl_settings = {
    "iteration_timeout_minutes": 5, # 各試行の最大実験時間
    "iterations": 10,  # 試行する機械学習パイプラインの数
    #"max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 3,
    "primary_metric": 'normalized_mean_absolute_error', # 精度指標
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'regression',
                             training_data = train_dataset,
                             label_column_name = label,                             
                             #compute_target=compute_target,
                             model_explainability = True,
                             **automl_settings
                            )

### 実行と結果確認

Machine Learning Compute で一番最初に実行する際は Docker Image を作成するため、20 〜 30分ほど時間がかかります。Dokcer Image を Build している様子は Azure Machine Learning studio から確認できます。

In [ ]:
remote_run = experiment.submit(automl_config, show_output = True)

## 結果の確認

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
# 詳細ログの出力
#remote_run.get_details()

In [ ]:
#remote_run.get_guardrails()

In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run

### 機械学習モデルの理解

In [ ]:
#pd.DataFrame(fitted_model.named_steps['datatransformer'].get_engineered_feature_names())

In [ ]:
#pd.DataFrame.from_records(fitted_model.named_steps['datatransformer'].get_featurization_summary())

## 3. モデル解釈
Azure Machine Learning には Automated ML のモデルを解釈する仕組みがあります。詳しくは [モデルを解釈する方法](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-machine-learning-interpretability#how-to-interpret-your-model)をご参照ください。

In [ ]:
# Pandas Dataframe に変換
train_df = train_dataset.to_pandas_dataframe()
test_df = test_dataset.to_pandas_dataframe()

In [ ]:
from azureml.train.automl.runtime.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations
automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, 
                                                             X=train_df.drop([label], axis=1), 
                                                             X_test=test_df.drop([label], axis=1), 
                                                             y=train_df[label].values, 
                                                             task='regression')

`MimicWrapper` は **Global Surrogaete Model** によるグローバルなモデル解釈を実現します。ここでは LightGBM を用いています。

In [ ]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

## Engineered Explanation
特徴量エンジニアリングで生成された変数を用いたモデル解釈を行います。

In [ ]:
# Compute the engineered explanations
engineered_explanations = explainer.explain(['local', 'global'],get_raw=False,
                                            eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [ ]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(engineered_explanations, 
                     automl_explainer_setup_obj.automl_estimator, 
                     datasetX=automl_explainer_setup_obj.X_test_transform)

## Raw Explanation
特徴量エンジニアリング前の変数を用いたモデル解釈を行います。

In [ ]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)

In [ ]:
# ダッシュボードの表示
from interpret_community.widget import ExplanationDashboard
ExplanationDashboard(raw_explanations, 
                     automl_explainer_setup_obj.automl_pipeline, 
                     datasetX=automl_explainer_setup_obj.X_test_raw)